Charger les variables :

In [8]:
%store -r RG_NAME ADLS_NAME ADLS_LOCATION

Créer le data lake :

In [ ]:
%%bash -s "$RG_NAME" "$ADLS_NAME" "$ADLS_LOCATION"

az storage account create \
      --name "$2" \
      --resource-group "$1" \
      --location "$3" \
      --sku Standard_LRS \
      --kind StorageV2 \
      --hns true # 🚨 
      --query "provisioningState" \
      --output tsv

In [ ]:
Vérifier que 

In [11]:
%%bash -s "$RG_NAME" "$ADLS_NAME"

KEYS=$(az storage account keys list -g "$1" -n "$2" --query "[].value" -o tsv)

# Extraire la première et la deuxième clé
KEY1=$(echo "$KEYS" | head -n1)
KEY2=$(echo "$KEYS" | tail -n1)

{
    echo "ADLS_KEY_1=\"$KEY1\""
    echo "ADLS_KEY_2=\"$KEY2\""
    echo ""
} >> .env

Créer les conteneurs :

In [12]:
%%bash -s $ADLS_NAME
source .env

for container in "logs" "region-ara" "region-hdf" "region-idf" "region-occ" "region-paca"
do 
    az storage container create  \
        --account-name "$1" \
        --name "$container" \
        --account-key $ADLS_KEY_1 \
        --output none
done

Charger les conteneurs avec les fichiers correspondants :

In [14]:
%%bash -s $ADLS_NAME
source .env
for container in "logs" "region-ara" "region-hdf" "region-idf" "region-occ" "region-paca"
do
    echo "Upload du dossier $container..."
    az storage blob upload-batch \
        --account-name "$1" \
        --account-key "$ADLS_KEY_1" \
        --destination "$container" \
        --source "DATA/Data_Lake/$container" \
        --pattern "*" \
        --output none
    echo "✅ Upload terminé pour $container"
done

Upload du dossier logs...


Finished[#############################################################]  100.0000%


✅ Upload terminé pour logs
Upload du dossier region-ara...


Finished[#############################################################]  100.0000%


✅ Upload terminé pour region-ara
Upload du dossier region-hdf...


...

✅ Upload terminé pour region-idf
Upload du dossier region-occ...


Finished[#############################################################]  100.0000%


✅ Upload terminé pour region-occ
Upload du dossier region-paca...


Finished[#############################################################]  100.0000%


✅ Upload terminé pour region-paca
